In [1]:
import pandas as pd
import xgboost
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import explained_variance_score
from sklearn.metrics import mean_absolute_error
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit
import pickle


In [2]:
case_rainfall_features = pd.read_csv('/Users/Rohil/Documents/iGEM/yemen/full_feature_data.csv')
case_rainfall_features.drop('weekly_cases', axis = 1, inplace = True)

In [4]:
fatality_features = pd.read_csv('/Users/Rohil/Documents/iGEM/yemen/full_fatality_features.csv')

In [5]:
y_df = pd.read_csv('/Users/Rohil/Documents/iGEM/yemen/cholera_epi_data/y_normalized_by_week_df.csv')

In [6]:
y_df.head()

,gov_iso,date,next_week_cases,next_two_week_cases,next_three_week_cases,next_four_week_cases,next_five_week_cases,next_six_week_cases
0,YE-AB,2017-05-23,6.976066,7.594340,8.919934,9.562493,11.945527,39.312472
1,YE-AB,2017-05-24,7.085352,7.853387,9.107136,8.166066,18.274987,37.308902
2,YE-AB,2017-05-25,7.194638,8.405887,9.177970,11.480051,17.424988,38.098187
3,YE-AB,2017-05-26,7.303923,8.958386,9.248803,11.773503,19.033915,38.948186
4,YE-AB,2017-05-27,7.413209,9.510886,9.319636,13.038384,21.022306,37.541640


In [7]:
case_rainfall_features = case_rainfall_features[(case_rainfall_features['date'] > '2017-05-30')].sort_values('date')

In [8]:
full_features = case_rainfall_features.merge(conflict_event_features.drop('weekly_cases', axis=1), on = ['gov_iso', 'date'], how = 'left').merge(fatality_features.drop('fatalities', axis=1), on = ['gov_iso', 'date'], how = 'left').fillna(0)

In [9]:
full_features.date = pd.to_datetime(full_features.date, format = '%Y-%m-%d')

In [10]:
full_features = pd.concat([full_features, pd.get_dummies(full_features.gov_iso)], axis=1)

In [11]:
full_features.head()

,gov_iso,date,mean_past_week_cases,max_past_week_cases,kurtosis_past_week_cases,mean_past_2_week_cases,max_past_2_week_cases,kurtosis_past_2_week_cases,mean_past_month_cases,max_past_month_cases,...,YE-LA,YE-MA,YE-MR,YE-MW,YE-RA,YE-SA,YE-SD,YE-SH,YE-SN,YE-TA
0,YE-AB,2017-05-31,0.996581,1.123213,-1.916667,0.984709,1.123213,-1.733333,0.984709,1.123213,...,0,0,0,0,0,0,0,0,0,0
1,YE-SN,2017-05-31,1.313100,1.375641,-1.916667,1.320918,1.375641,-1.733333,1.320918,1.375641,...,0,0,0,0,0,0,0,0,1,0
2,YE-SH,2017-05-31,0.006578,0.015349,-1.916667,0.005756,0.015349,-1.733333,0.005756,0.015349,...,0,0,0,0,0,0,0,1,0,0
3,YE-BA,2017-05-31,2.445220,2.484196,-1.916667,2.450092,2.484196,-1.733333,2.450092,2.484196,...,0,0,0,0,0,0,0,0,0,0
4,YE-IB,2017-05-31,0.375929,0.408584,-1.916667,0.380011,0.408584,-1.733333,0.380011,0.408584,...,0,0,0,0,0,0,0,0,0,0


In [12]:
#full_features.to_csv('/Users/Rohil/Documents/iGEM/yemen/full_features.csv', index = False)

In [13]:
full_features.iloc[:,164:185].shape

(5397, 21)

In [14]:
categorical_cols = (162, 183)

In [15]:
all_feature_list = full_features.drop('gov_iso', axis = 1).columns

In [16]:
full_features.date = pd.to_datetime(full_features.date, format = '%Y-%m-%d')
y_df.date = pd.to_datetime(y_df.date, format = '%Y-%m-%d')

In [17]:
full_features

,gov_iso,date,mean_past_week_cases,max_past_week_cases,kurtosis_past_week_cases,mean_past_2_week_cases,max_past_2_week_cases,kurtosis_past_2_week_cases,mean_past_month_cases,max_past_month_cases,...,YE-LA,YE-MA,YE-MR,YE-MW,YE-RA,YE-SA,YE-SD,YE-SH,YE-SN,YE-TA
0,YE-AB,2017-05-31,0.996581,1.123213,-1.916667,0.984709,1.123213,-1.733333,0.984709,1.123213,...,0,0,0,0,0,0,0,0,0,0
1,YE-SN,2017-05-31,1.313100,1.375641,-1.916667,1.320918,1.375641,-1.733333,1.320918,1.375641,...,0,0,0,0,0,0,0,0,1,0
2,YE-SH,2017-05-31,0.006578,0.015349,-1.916667,0.005756,0.015349,-1.733333,0.005756,0.015349,...,0,0,0,0,0,0,0,1,0,0
3,YE-BA,2017-05-31,2.445220,2.484196,-1.916667,2.450092,2.484196,-1.733333,2.450092,2.484196,...,0,0,0,0,0,0,0,0,0,0
4,YE-IB,2017-05-31,0.375929,0.408584,-1.916667,0.380011,0.408584,-1.733333,0.380011,0.408584,...,0,0,0,0,0,0,0,0,0,0
5,YE-SA,2017-05-31,4.643352,5.665804,-1.916667,4.771159,5.665804,-1.733333,4.771159,5.665804,...,0,0,0,0,0,1,0,0,0,0
6,YE-DA,2017-05-31,2.053912,2.500223,-1.916667,2.012071,2.500223,-1.733333,2.012071,2.500223,...,0,0,0,0,0,0,0,0,0,0
7,YE-SD,2017-05-31,0.028928,0.067499,-1.916667,0.025312,0.067499,-1.733333,0.025312,0.067499,...,0,0,0,0,0,0,1,0,0,0
8,YE-MW,2017-05-31,3.000866,3.025834,-1.916667,3.003987,3.025834,-1.733333,3.003987,3.025834,...,0,0,0,1,0,0,0,0,0,0
9,YE-TA,2017-05-31,0.717478,0.735545,-1.916667,0.715785,0.735545,-1.733333,0.715785,0.735545,...,0,0,0,0,0,0,0,0,0,1


In [18]:
full_data = full_features.merge(y_df, on = ['gov_iso', 'date'], how = 'left')

In [19]:
X = full_data[all_feature_list]

In [20]:
y1 = full_data[['date', 'next_week_cases']]
y2 = full_data[['date', 'next_two_week_cases']]
y3 = full_data[['date', 'next_three_week_cases']]
y4 = full_data[['date', 'next_four_week_cases']]
y5 = full_data[['date', 'next_five_week_cases']]
y6 = full_data[['date', 'next_six_week_cases']]

In [21]:
def scale_features(scaler, X, categorical_cols):
    
    X_numerical = X.iloc[:,:(categorical_cols[0])]
    X_categorical = X.iloc[:, categorical_cols[0]:categorical_cols[1]]
        
    X_numerical_scaled = pd.DataFrame(data=scaler.transform(X_numerical), columns = X_numerical.columns)
    
    X_scaled = pd.concat([X_numerical_scaled, X_categorical.reset_index(drop=True)], axis=1)
    
    return (X_scaled)

In [22]:
def train_base_selection_model(X, y, param_grid, split_date, categorical_cols):
    
    # some of the ending values are undefined purposely, as there isnt't enough data to get cholera cases for x weeks
    y = y.dropna()
    X = X.loc[y.index]
    
    print ('X shape:')
    
    X_train, X_test = X[(X.date<split_date)].drop('date', axis=1), X[(X.date>=split_date)].drop('date', axis=1)
    y_train, y_test = y[(y.date<split_date)].drop('date', axis=1), y[(y.date>=split_date)].drop('date', axis=1)
    
    scaler = StandardScaler()
    scaler.fit(X_train.iloc[:,:categorical_cols[0]])
    
    X_train, X_test = scale_features(scaler, X_train, categorical_cols), scale_features(scaler, X_test, categorical_cols)
    
    xgb = xgboost.XGBRegressor()
    
    grid_search = GridSearchCV(xgb,
                        param_grid = param_grid,
                        cv = TimeSeriesSplit(n_splits=2).split(X_train),
                        n_jobs = -1,
                        scoring = 'explained_variance',
                        verbose=True)
    
    grid_search.fit(X_train, y_train)
    
    y_pred = grid_search.predict(X_test)
    score = explained_variance_score(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    
    print('w/ all features... explained variance: %s, mean abs error: %s' % (score, mae))

    
    return(grid_search.best_estimator_, grid_search.best_params_)

In [23]:
base_param_grid = {'nthread':[4], #when use hyperthread, xgboost may become slower
              'objective':['reg:linear'],
              'learning_rate': [0.01], #so called `eta` value
              'max_depth': [10, 15, 20],
              'min_child_weight': [15, 30, 45],
              'silent': [1],
              'subsample': [0.4, 0.6, 0.8],
              'colsample_bytree': [0.4, 0.6, 0.8],
              'n_estimators': [150, 250, 350]}

In [24]:
week1_xgb, week1_params = train_base_selection_model(X, y1, base_param_grid, '2017-09-25', categorical_cols)

X shape:
Fitting 2 folds for each of 243 candidates, totalling 486 fits


[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   22.9s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:  1.9min
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:  5.5min
[Parallel(n_jobs=-1)]: Done 486 out of 486 | elapsed:  6.4min finished


w/ all features... explained variance: 0.8286793494376657, mean abs error: 1.5383716653625767


In [25]:
week2_xgb, week2_params = train_base_selection_model(X, y2, base_param_grid, '2017-09-25', categorical_cols)

X shape:
Fitting 2 folds for each of 243 candidates, totalling 486 fits


[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   24.7s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:  2.0min
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:  5.7min
[Parallel(n_jobs=-1)]: Done 486 out of 486 | elapsed:  6.7min finished


w/ all features... explained variance: 0.7553790910202517, mean abs error: 1.8808571368800777


In [26]:
week3_xgb, week3_params = train_base_selection_model(X, y3, base_param_grid, '2017-09-25', categorical_cols)

X shape:
Fitting 2 folds for each of 243 candidates, totalling 486 fits


[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   27.5s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:  2.2min
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:  6.2min
[Parallel(n_jobs=-1)]: Done 486 out of 486 | elapsed:  7.1min finished


w/ all features... explained variance: 0.7120905164559378, mean abs error: 2.1160826563969106


In [27]:
week4_xgb, week4_params = train_base_selection_model(X, y4, base_param_grid, '2017-09-25', categorical_cols)

X shape:
Fitting 2 folds for each of 243 candidates, totalling 486 fits


[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   24.8s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:  2.0min
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:  6.0min
[Parallel(n_jobs=-1)]: Done 486 out of 486 | elapsed:  6.9min finished


w/ all features... explained variance: 0.5564145534824849, mean abs error: 2.693501299084113


In [28]:
week5_xgb, week5_params = train_base_selection_model(X, y5, base_param_grid, '2017-09-25', categorical_cols)

X shape:
Fitting 2 folds for each of 243 candidates, totalling 486 fits


[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   26.6s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:  2.1min
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:  6.1min
[Parallel(n_jobs=-1)]: Done 486 out of 486 | elapsed:  7.0min finished


w/ all features... explained variance: 0.4899610228327774, mean abs error: 2.832220925610058


In [29]:
week6_xgb, week6_params = train_base_selection_model(X, y6, base_param_grid, '2017-09-25', categorical_cols)

X shape:
Fitting 2 folds for each of 243 candidates, totalling 486 fits


[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   26.6s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:  2.1min
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:  6.1min
[Parallel(n_jobs=-1)]: Done 486 out of 486 | elapsed:  7.1min finished


w/ all features... explained variance: 0.5468819141676542, mean abs error: 2.558438988464018


In [30]:
week1_params

{'colsample_bytree': 0.4,
 'learning_rate': 0.01,
 'max_depth': 10,
 'min_child_weight': 45,
 'n_estimators': 250,
 'nthread': 4,
 'objective': 'reg:linear',
 'silent': 1,
 'subsample': 0.4}

In [31]:
week2_params

{'colsample_bytree': 0.6,
 'learning_rate': 0.01,
 'max_depth': 10,
 'min_child_weight': 45,
 'n_estimators': 150,
 'nthread': 4,
 'objective': 'reg:linear',
 'silent': 1,
 'subsample': 0.4}

In [33]:
week3_params

{'colsample_bytree': 0.4,
 'learning_rate': 0.01,
 'max_depth': 10,
 'min_child_weight': 45,
 'n_estimators': 150,
 'nthread': 4,
 'objective': 'reg:linear',
 'silent': 1,
 'subsample': 0.4}

In [32]:
week4_params

{'colsample_bytree': 0.4,
 'learning_rate': 0.01,
 'max_depth': 10,
 'min_child_weight': 15,
 'n_estimators': 150,
 'nthread': 4,
 'objective': 'reg:linear',
 'silent': 1,
 'subsample': 0.4}

In [35]:
week5_params

{'colsample_bytree': 0.4,
 'learning_rate': 0.01,
 'max_depth': 10,
 'min_child_weight': 15,
 'n_estimators': 150,
 'nthread': 4,
 'objective': 'reg:linear',
 'silent': 1,
 'subsample': 0.4}

In [34]:
week6_params

{'colsample_bytree': 0.4,
 'learning_rate': 0.01,
 'max_depth': 15,
 'min_child_weight': 15,
 'n_estimators': 150,
 'nthread': 4,
 'objective': 'reg:linear',
 'silent': 1,
 'subsample': 0.8}

In [36]:
model_list = [week1_xgb, week2_xgb, week3_xgb, week4_xgb, week5_xgb, week6_xgb]

In [39]:
for i in range(0, len(model_list)):
    fig, ax = plt.subplots(1,1, figsize = (10, 30))
    xgboost.plot_importance(model_list[i], ax=ax)
    fig.tight_layout()
    fig.savefig('/Users/Rohil/Documents/iGEM/yemen/feature_engineering/xgb_' + str(i + 1) + '_week_only_feature_importance.png')
    plt.clf()
    plt.close('all')

In [47]:
nfeature_param_grid = {'nthread':[4], #when use hyperthread, xgboost may become slower
              'objective':['reg:linear'],
              'learning_rate': [0.01], #so called `eta` value
              'max_depth': [10, 15],
              'min_child_weight': [15, 30, 45],
              'silent': [1],
              'subsample': [0.5, 0.65, 0.8],
              'colsample_bytree': [0.5, 0.65, 0.8],
              'n_estimators': [100, 200],
}

In [48]:
def nfeature_gridsearch(X, y, split_date, xgb_all_features, param_grid, categorical_cols):
    
    thresholds = sorted(xgb_all_features.feature_importances_, reverse = True)
    five_thresh = thresholds[0:100][0::5]
    
    nfeature_gridsearch_df = pd.DataFrame(columns=['threshold', 'n_features', 'best_params', 'explained_variance', 'mae'])

    # some of the ending values are undefined purposely, as there isnt't enough data to get cholera cases for x weeks
    y = y.dropna()
    X = X.loc[y.index]
        
    X_train, X_test = X[(X.date<split_date)].drop('date', axis=1), X[(X.date>=split_date)].drop('date', axis=1)
    y_train, y_test = y[(y.date<split_date)].drop('date', axis=1), y[(y.date>=split_date)].drop('date', axis=1)
    
    scaler = StandardScaler()
    scaler.fit(X_train.iloc[:,:categorical_cols[0]])
    
    X_train, X_test = scale_features(scaler, X_train, categorical_cols), scale_features(scaler, X_test, categorical_cols)
    
    for thresh in thresholds[0:100]:
        # select features using threshold
        selection = SelectFromModel(xgb_all_features, threshold=thresh, prefit=True)

        select_X_train = selection.transform(X_train)
        
        selection_model = xgboost.XGBRegressor()

        grid_search = GridSearchCV(selection_model,
                            param_grid = param_grid,
                            cv = TimeSeriesSplit(n_splits=2).split(select_X_train),
                            n_jobs = -1,
                            scoring = 'explained_variance',
                            verbose=True)


        grid_search.fit(select_X_train, y_train)

        # eval model
        select_X_test = selection.transform(X_test)
        y_pred = grid_search.predict(select_X_test)
        score = explained_variance_score(y_test, y_pred)
        mae = mean_absolute_error(y_test, y_pred)

        print('Thresh=%s, n=%s, explained variance: %s, mean abs error: %s' % (thresh, select_X_train.shape[1], score, mae))

        nfeature_gridsearch_df = nfeature_gridsearch_df.append(pd.Series({'threshold':thresh, 'n_features':select_X_train.shape[1], 'best_params': grid_search.best_params_, 'explained_variance':score, 'mae': mae}), ignore_index = True)
        
    return (nfeature_gridsearch_df)

In [49]:
week1_nfeature_gridsearch = nfeature_gridsearch(X, y1, '2017-09-25', week1_xgb, nfeature_param_grid, categorical_cols)

Fitting 2 folds for each of 108 candidates, totalling 216 fits


[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    6.2s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:   14.6s
[Parallel(n_jobs=-1)]: Done 216 out of 216 | elapsed:   16.5s finished


Thresh=0.09749232, n=1, explained variance: 0.8303297027178167, mean abs error: 1.405146565511924
Fitting 2 folds for each of 108 candidates, totalling 216 fits


[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    6.3s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:   14.4s
[Parallel(n_jobs=-1)]: Done 216 out of 216 | elapsed:   16.3s finished


Thresh=0.07676561, n=2, explained variance: 0.8285179794953167, mean abs error: 1.4480368395374263
Fitting 2 folds for each of 108 candidates, totalling 216 fits


...

Thresh=0.0005117707, n=104, explained variance: 0.8419448303332037, mean abs error: 1.4236275579971638
Fitting 2 folds for each of 108 candidates, totalling 216 fits


[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   12.5s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:   56.4s
[Parallel(n_jobs=-1)]: Done 216 out of 216 | elapsed:  1.1min finished


Thresh=0.0005117707, n=104, explained variance: 0.8419448303332037, mean abs error: 1.4236275579971638
Fitting 2 folds for each of 108 candidates, totalling 216 fits


[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   12.9s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:   57.5s
[Parallel(n_jobs=-1)]: Done 216 out of 216 | elapsed:  1.1min finished


Thresh=0.0005117707, n=104, explained variance: 0.8419448303332037, mean abs error: 1.4236275579971638


In [50]:
week2_nfeature_gridsearch = nfeature_gridsearch(X, y2, '2017-09-25', week2_xgb, nfeature_param_grid, categorical_cols)

Fitting 2 folds for each of 108 candidates, totalling 216 fits


[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    5.9s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:   13.7s
[Parallel(n_jobs=-1)]: Done 216 out of 216 | elapsed:   15.5s finished


Thresh=0.14470842, n=1, explained variance: 0.8094797633990592, mean abs error: 3.211365934950691
Fitting 2 folds for each of 108 candidates, totalling 216 fits


[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    6.0s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:   13.8s
[Parallel(n_jobs=-1)]: Done 216 out of 216 | elapsed:   15.6s finished


Thresh=0.09719223, n=2, explained variance: 0.8168454050906664, mean abs error: 3.2561631977784846
Fitting 2 folds for each of 108 candidates, totalling 216 fits


...

Thresh=0.0, n=183, explained variance: 0.8148520588735845, mean abs error: 3.289227270358747
Fitting 2 folds for each of 108 candidates, totalling 216 fits


[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   15.9s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done 216 out of 216 | elapsed:  1.6min finished


Thresh=0.0, n=183, explained variance: 0.8148520588735845, mean abs error: 3.289227270358747
Fitting 2 folds for each of 108 candidates, totalling 216 fits


[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   16.6s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done 216 out of 216 | elapsed:  1.6min finished


Thresh=0.0, n=183, explained variance: 0.8148520588735845, mean abs error: 3.289227270358747


In [51]:
week4_nfeature_gridsearch = nfeature_gridsearch(X, y4, '2017-09-25', week4_xgb, nfeature_param_grid, categorical_cols)

Fitting 2 folds for each of 108 candidates, totalling 216 fits


[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    5.9s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:   13.5s
[Parallel(n_jobs=-1)]: Done 216 out of 216 | elapsed:   15.3s finished


Thresh=0.103797466, n=1, explained variance: 0.7647351997584207, mean abs error: 8.150259185014216
Fitting 2 folds for each of 108 candidates, totalling 216 fits


[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    6.0s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:   13.8s
[Parallel(n_jobs=-1)]: Done 216 out of 216 | elapsed:   15.6s finished


Thresh=0.067510545, n=2, explained variance: 0.7586334040176724, mean abs error: 8.409207483857228
Fitting 2 folds for each of 108 candidates, totalling 216 fits


...

Thresh=0.00042194093, n=108, explained variance: 0.722195269258969, mean abs error: 9.344307361062603
Fitting 2 folds for each of 108 candidates, totalling 216 fits


[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   12.3s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:   58.2s
[Parallel(n_jobs=-1)]: Done 216 out of 216 | elapsed:  1.1min finished


Thresh=0.00042194093, n=108, explained variance: 0.722195269258969, mean abs error: 9.344307361062603
Fitting 2 folds for each of 108 candidates, totalling 216 fits


[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   12.7s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:   58.5s
[Parallel(n_jobs=-1)]: Done 216 out of 216 | elapsed:  1.2min finished


Thresh=0.00042194093, n=108, explained variance: 0.722195269258969, mean abs error: 9.344307361062603


In [52]:
week6_nfeature_gridsearch = nfeature_gridsearch(X, y6, '2017-09-25', week6_xgb, nfeature_param_grid, categorical_cols)

Fitting 2 folds for each of 108 candidates, totalling 216 fits


[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    6.0s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:   13.7s
[Parallel(n_jobs=-1)]: Done 216 out of 216 | elapsed:   15.6s finished


Thresh=0.0321321, n=1, explained variance: 0.7396234140455316, mean abs error: 12.43686032115643
Fitting 2 folds for each of 108 candidates, totalling 216 fits


[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    6.2s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:   15.0s
[Parallel(n_jobs=-1)]: Done 216 out of 216 | elapsed:   17.1s finished


Thresh=0.031239541, n=2, explained variance: 0.715546287795185, mean abs error: 15.33558335064881
Fitting 2 folds for each of 108 candidates, totalling 216 fits


...

Thresh=0.0010041281, n=98, explained variance: 0.636848883162913, mean abs error: 15.021152496579163
Fitting 2 folds for each of 108 candidates, totalling 216 fits


[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   12.6s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:   58.9s
[Parallel(n_jobs=-1)]: Done 216 out of 216 | elapsed:  1.2min finished


Thresh=0.0008925583, n=107, explained variance: 0.6142501217456602, mean abs error: 16.127031449476128
Fitting 2 folds for each of 108 candidates, totalling 216 fits


[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   12.7s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:   58.8s
[Parallel(n_jobs=-1)]: Done 216 out of 216 | elapsed:  1.2min finished


Thresh=0.0008925583, n=107, explained variance: 0.6142501217456602, mean abs error: 16.127031449476128


In [49]:
pickle.dump(week1_xgb, open("/Users/Rohil/Documents/iGEM/yemen/feature_engineering/week_only_models/week1_xgb_all_features_week_only.pickle.dat", "wb"))
pickle.dump(week2_xgb, open("/Users/Rohil/Documents/iGEM/yemen/feature_engineering/week_only_models/week2_xgb_all_features_week_only.pickle.dat", "wb"))
pickle.dump(week3_xgb, open("/Users/Rohil/Documents/iGEM/yemen/feature_engineering/week_only_models/week3_xgb_all_features_week_only.pickle.dat", "wb"))
pickle.dump(week4_xgb, open("/Users/Rohil/Documents/iGEM/yemen/feature_engineering/week_only_models/week4_xgb_all_features_week_only.pickle.dat", "wb"))
pickle.dump(week5_xgb, open("/Users/Rohil/Documents/iGEM/yemen/feature_engineering/week_only_models/week5_xgb_all_features_week_only.pickle.dat", "wb"))
pickle.dump(week6_xgb, open("/Users/Rohil/Documents/iGEM/yemen/feature_engineering/week_only_models/week6_xgb_all_features_week_only.pickle.dat", "wb"))

In [54]:
week1_nfeature_gridsearch.to_csv('/Users/Rohil/Documents/iGEM/yemen/feature_engineering/week1_nfeatures_gridsearch_df.csv', index = False)
week2_nfeature_gridsearch.to_csv('/Users/Rohil/Documents/iGEM/yemen/feature_engineering/week2_nfeatures_gridsearch_df.csv', index = False)
week4_nfeature_gridsearch.to_csv('/Users/Rohil/Documents/iGEM/yemen/feature_engineering/week4_nfeatures_gridsearch_df.csv', index = False)
week6_nfeature_gridsearch.to_csv('/Users/Rohil/Documents/iGEM/yemen/feature_engineering/week6_nfeatures_gridsearch_df.csv', index = False)

In [40]:
def nfeature_search(X, y, split_date, xgb_all_features, model_params, categorical_cols):
    
    thresholds = sorted(xgb_all_features.feature_importances_, reverse = True)
    
    nfeature_df = pd.DataFrame(columns=['threshold', 'n_features', 'explained_variance', 'mae'])

    # some of the ending values are undefined purposely, as there isnt't enough data to get cholera cases for x weeks
    y = y.dropna()
    X = X.loc[y.index]
        
    X_train, X_test = X[(X.date<split_date)].drop('date', axis=1), X[(X.date>=split_date)].drop('date', axis=1)
    y_train, y_test = y[(y.date<split_date)].drop('date', axis=1), y[(y.date>=split_date)].drop('date', axis=1)
    
    scaler = StandardScaler()
    scaler.fit(X_train.iloc[:,:categorical_cols[0]])
    
    X_train, X_test = scale_features(scaler, X_train, categorical_cols), scale_features(scaler, X_test, categorical_cols)
    
    for thresh in thresholds[0:100]:
        # select features using threshold
        selection = SelectFromModel(xgb_all_features, threshold=thresh, prefit=True)

        select_X_train = selection.transform(X_train)
        
        selection_model = xgboost.XGBRegressor(**model_params)

        selection_model.fit(select_X_train, y_train)

        # eval model
        select_X_test = selection.transform(X_test)
        y_pred = selection_model.predict(select_X_test)
        score = explained_variance_score(y_test, y_pred)
        mae = mean_absolute_error(y_test, y_pred)

        print('Thresh=%s, n=%s, explained variance: %s, mean abs error: %s' % (thresh, select_X_train.shape[1], score, mae))

        nfeature_df = nfeature_df.append(pd.Series({'threshold':thresh, 'n_features':select_X_train.shape[1], 'explained_variance':score, 'mae': mae}), ignore_index = True)
        
    return (nfeature_df)

In [41]:
week1_nfeature_search = nfeature_search(X, y1, '2017-09-25', week1_xgb, week1_params, categorical_cols)

Thresh=0.09749232, n=1, explained variance: 0.8211068483966901, mean abs error: 1.5391319928360536
Thresh=0.07676561, n=2, explained variance: 0.8151773559913033, mean abs error: 1.6094321984413764
Thresh=0.042732857, n=3, explained variance: 0.8153736614121094, mean abs error: 1.902614090949244
Thresh=0.035056293, n=4, explained variance: 0.7972784735754975, mean abs error: 1.7959587475122576
Thresh=0.032753326, n=5, explained variance: 0.8129667065764891, mean abs error: 1.587240356042348
Thresh=0.025588537, n=6, explained variance: 0.8291892621480452, mean abs error: 1.3824349057477607
Thresh=0.02533265, n=7, explained variance: 0.817887415492959, mean abs error: 1.4502776121657337
Thresh=0.022773797, n=8, explained variance: 0.8288576947000662, mean abs error: 1.4580146485324528
Thresh=0.022006141, n=9, explained variance: 0.8329527453770288, mean abs error: 1.427700175788198
Thresh=0.021238485, n=10, explained variance: 0.8282299954515931, mean abs error: 1.5270805005555397
Thresh

Thresh=0.0007676561, n=88, explained variance: 0.829172605484068, mean abs error: 1.517049050622963
Thresh=0.0007676561, n=88, explained variance: 0.829172605484068, mean abs error: 1.517049050622963
Thresh=0.0007676561, n=88, explained variance: 0.829172605484068, mean abs error: 1.517049050622963
Thresh=0.0007676561, n=88, explained variance: 0.829172605484068, mean abs error: 1.517049050622963
Thresh=0.0007676561, n=88, explained variance: 0.829172605484068, mean abs error: 1.517049050622963
Thresh=0.0007676561, n=88, explained variance: 0.829172605484068, mean abs error: 1.517049050622963
Thresh=0.0005117707, n=104, explained variance: 0.825766027165283, mean abs error: 1.571066295074216
Thresh=0.0005117707, n=104, explained variance: 0.825766027165283, mean abs error: 1.571066295074216
Thresh=0.0005117707, n=104, explained variance: 0.825766027165283, mean abs error: 1.571066295074216
Thresh=0.0005117707, n=104, explained variance: 0.825766027165283, mean abs error: 1.571066295074

In [42]:
week2_nfeature_search = nfeature_search(X, y2, '2017-09-25', week2_xgb, week2_params, categorical_cols)

Thresh=0.11498708, n=1, explained variance: 0.7676571276778512, mean abs error: 1.7507212179185592
Thresh=0.08699397, n=2, explained variance: 0.7713009216273319, mean abs error: 1.7893305740044456
Thresh=0.05641688, n=3, explained variance: 0.7753437319269085, mean abs error: 2.0097988273522343
Thresh=0.04909561, n=4, explained variance: 0.7493113170321504, mean abs error: 2.0085341343014984
Thresh=0.03057709, n=5, explained variance: 0.7538295404949433, mean abs error: 1.852737056649311
Thresh=0.030146426, n=6, explained variance: 0.7812892628238577, mean abs error: 1.7390367194042962
Thresh=0.0292851, n=7, explained variance: 0.7749816027251356, mean abs error: 1.6577842685404098
Thresh=0.023255814, n=8, explained variance: 0.7729486149750333, mean abs error: 1.6657855762930174
Thresh=0.021963824, n=9, explained variance: 0.7755662397017117, mean abs error: 1.690518721205123
Thresh=0.021102497, n=10, explained variance: 0.7680319292861438, mean abs error: 1.7097655665177105
Thresh=0

Thresh=0.00086132647, n=85, explained variance: 0.7572709660698083, mean abs error: 1.8613408344779254
Thresh=0.00086132647, n=85, explained variance: 0.7572709660698083, mean abs error: 1.8613408344779254
Thresh=0.00086132647, n=85, explained variance: 0.7572709660698083, mean abs error: 1.8613408344779254
Thresh=0.00043066323, n=103, explained variance: 0.7660009642401395, mean abs error: 1.8003906876266988
Thresh=0.00043066323, n=103, explained variance: 0.7660009642401395, mean abs error: 1.8003906876266988
Thresh=0.00043066323, n=103, explained variance: 0.7660009642401395, mean abs error: 1.8003906876266988
Thresh=0.00043066323, n=103, explained variance: 0.7660009642401395, mean abs error: 1.8003906876266988
Thresh=0.00043066323, n=103, explained variance: 0.7660009642401395, mean abs error: 1.8003906876266988
Thresh=0.00043066323, n=103, explained variance: 0.7660009642401395, mean abs error: 1.8003906876266988
Thresh=0.00043066323, n=103, explained variance: 0.7660009642401395

In [44]:
week3_nfeature_search = nfeature_search(X, y3, '2017-09-25', week3_xgb, week3_params, categorical_cols)

Thresh=0.07453936, n=1, explained variance: 0.7098051098678697, mean abs error: 2.1775058544917063
Thresh=0.0519263, n=2, explained variance: 0.7011785945616734, mean abs error: 2.2469125942181325
Thresh=0.051507536, n=3, explained variance: 0.6886723584677386, mean abs error: 2.54699878812228
Thresh=0.04438861, n=4, explained variance: 0.6331859723014117, mean abs error: 3.0945753018019424
Thresh=0.03517588, n=5, explained variance: 0.7350957490849703, mean abs error: 2.0588750516759444
Thresh=0.03475712, n=6, explained variance: 0.7301671796876383, mean abs error: 2.2348560950489573
Thresh=0.028894473, n=7, explained variance: 0.7149879334382118, mean abs error: 2.004996153420403
Thresh=0.023450587, n=8, explained variance: 0.7246492917438709, mean abs error: 1.9289774310323722
Thresh=0.022613065, n=9, explained variance: 0.7188960817984984, mean abs error: 1.9310485385617695
Thresh=0.022194305, n=10, explained variance: 0.7038641514027124, mean abs error: 2.0070837664489414
Thresh=0

Thresh=0.0012562814, n=86, explained variance: 0.7068379421322218, mean abs error: 2.133716272640504
Thresh=0.0012562814, n=86, explained variance: 0.7068379421322218, mean abs error: 2.133716272640504
Thresh=0.0012562814, n=86, explained variance: 0.7068379421322218, mean abs error: 2.133716272640504
Thresh=0.0012562814, n=86, explained variance: 0.7068379421322218, mean abs error: 2.133716272640504
Thresh=0.0008375209, n=98, explained variance: 0.7111349938328558, mean abs error: 2.1917855905924406
Thresh=0.0008375209, n=98, explained variance: 0.7111349938328558, mean abs error: 2.1917855905924406
Thresh=0.0008375209, n=98, explained variance: 0.7111349938328558, mean abs error: 2.1917855905924406
Thresh=0.0008375209, n=98, explained variance: 0.7111349938328558, mean abs error: 2.1917855905924406
Thresh=0.0008375209, n=98, explained variance: 0.7111349938328558, mean abs error: 2.1917855905924406
Thresh=0.0008375209, n=98, explained variance: 0.7111349938328558, mean abs error: 2.1

In [45]:
week4_nfeature_search = nfeature_search(X, y4, '2017-10-16', week4_xgb, week4_params, categorical_cols)

Thresh=0.035664227, n=1, explained variance: 0.3169491966127601, mean abs error: 3.0190876412511574
Thresh=0.030992057, n=2, explained variance: 0.4981440477360157, mean abs error: 2.5807822008721475
Thresh=0.030057624, n=3, explained variance: 0.586773629003098, mean abs error: 2.2497812797112897
Thresh=0.028033016, n=4, explained variance: 0.6193060059514004, mean abs error: 1.9932590867283284
Thresh=0.025229715, n=6, explained variance: 0.6469363980326052, mean abs error: 1.8499658741144793
Thresh=0.025229715, n=6, explained variance: 0.6469363980326052, mean abs error: 1.8499658741144793
Thresh=0.025073975, n=7, explained variance: 0.594806865415695, mean abs error: 2.1111466551417117
Thresh=0.024762498, n=8, explained variance: 0.6339902709316569, mean abs error: 1.789178482611559
Thresh=0.022114936, n=9, explained variance: 0.6274952110920791, mean abs error: 1.8408127413972
Thresh=0.020869024, n=10, explained variance: 0.6241037368191551, mean abs error: 1.8172413003315344
Thres

Thresh=0.0017131288, n=87, explained variance: 0.4527901498621023, mean abs error: 2.312062967850082
Thresh=0.0017131288, n=87, explained variance: 0.4527901498621023, mean abs error: 2.312062967850082
Thresh=0.0017131288, n=87, explained variance: 0.4527901498621023, mean abs error: 2.312062967850082
Thresh=0.0017131288, n=87, explained variance: 0.4527901498621023, mean abs error: 2.312062967850082
Thresh=0.0017131288, n=87, explained variance: 0.4527901498621023, mean abs error: 2.312062967850082
Thresh=0.0015573899, n=91, explained variance: 0.44253417951633345, mean abs error: 2.374943864570362
Thresh=0.0015573899, n=91, explained variance: 0.44253417951633345, mean abs error: 2.374943864570362
Thresh=0.0015573899, n=91, explained variance: 0.44253417951633345, mean abs error: 2.374943864570362
Thresh=0.0015573899, n=91, explained variance: 0.44253417951633345, mean abs error: 2.374943864570362
Thresh=0.0014016508, n=94, explained variance: 0.4664019658649937, mean abs error: 2.27

In [46]:
week5_nfeature_search = nfeature_search(X, y5, '2017-10-16', week5_xgb, week5_params, categorical_cols)

Thresh=0.033385094, n=1, explained variance: 0.1664574247829952, mean abs error: 3.1220819545003216
Thresh=0.032919254, n=2, explained variance: 0.5432507271344595, mean abs error: 2.0394633618917126
Thresh=0.032298137, n=3, explained variance: 0.54011650091582, mean abs error: 2.101565065319785
Thresh=0.030279502, n=4, explained variance: 0.6061243524780467, mean abs error: 1.8502221065120519
Thresh=0.02810559, n=5, explained variance: 0.6195384610443191, mean abs error: 1.6263015569630834
Thresh=0.023913043, n=6, explained variance: 0.6116673104996553, mean abs error: 1.762012169764974
Thresh=0.023291925, n=7, explained variance: 0.5720638568550181, mean abs error: 2.0244024713377904
Thresh=0.021428572, n=8, explained variance: 0.5046097028226592, mean abs error: 2.1106561809002544
Thresh=0.020652173, n=10, explained variance: 0.481750111652929, mean abs error: 2.08402965457489
Thresh=0.020652173, n=10, explained variance: 0.481750111652929, mean abs error: 2.08402965457489
Thresh=0.

Thresh=0.001863354, n=86, explained variance: 0.3742235407415032, mean abs error: 2.3040752162559
Thresh=0.001863354, n=86, explained variance: 0.3742235407415032, mean abs error: 2.3040752162559
Thresh=0.001863354, n=86, explained variance: 0.3742235407415032, mean abs error: 2.3040752162559
Thresh=0.001863354, n=86, explained variance: 0.3742235407415032, mean abs error: 2.3040752162559
Thresh=0.0017080746, n=88, explained variance: 0.4004301915340538, mean abs error: 2.2554868597333315
Thresh=0.0017080746, n=88, explained variance: 0.4004301915340538, mean abs error: 2.2554868597333315
Thresh=0.0015527951, n=91, explained variance: 0.4308182947827649, mean abs error: 2.207509261334268
Thresh=0.0015527951, n=91, explained variance: 0.4308182947827649, mean abs error: 2.207509261334268
Thresh=0.0015527951, n=91, explained variance: 0.4308182947827649, mean abs error: 2.207509261334268
Thresh=0.0013975155, n=93, explained variance: 0.3544310159035303, mean abs error: 2.340883590699356


In [47]:
week6_nfeature_search = nfeature_search(X, y6, '2017-10-16', week6_xgb, week6_params, categorical_cols)

Thresh=0.032733224, n=1, explained variance: 0.1663427745329944, mean abs error: 3.0387728386606234
Thresh=0.03265139, n=2, explained variance: 0.39660645771157543, mean abs error: 2.1822937418196457
Thresh=0.028559739, n=3, explained variance: 0.42603572983625526, mean abs error: 2.161537138723096
Thresh=0.02594108, n=4, explained variance: 0.4030345458984248, mean abs error: 2.6886233620133733
Thresh=0.025613748, n=5, explained variance: 0.2917712440804556, mean abs error: 2.643027796264802
Thresh=0.024549918, n=6, explained variance: 0.4058971690064713, mean abs error: 2.3667114290923856
Thresh=0.02299509, n=7, explained variance: 0.4837063497020264, mean abs error: 2.3226144532607034
Thresh=0.02193126, n=8, explained variance: 0.46661028985465036, mean abs error: 2.171373954914766
Thresh=0.02168576, n=9, explained variance: 0.5096381072747584, mean abs error: 2.012186855910831
Thresh=0.020785598, n=10, explained variance: 0.5204064979828221, mean abs error: 1.9109463525234303
Thres

Thresh=0.0019639935, n=84, explained variance: 0.5709507619938399, mean abs error: 1.6994429054890863
Thresh=0.0019639935, n=84, explained variance: 0.5709507619938399, mean abs error: 1.6994429054890863
Thresh=0.0018821604, n=88, explained variance: 0.5424295372540064, mean abs error: 1.7547951609752197
Thresh=0.0018821604, n=88, explained variance: 0.5424295372540064, mean abs error: 1.7547951609752197
Thresh=0.0018821604, n=88, explained variance: 0.5424295372540064, mean abs error: 1.7547951609752197
Thresh=0.0018821604, n=88, explained variance: 0.5424295372540064, mean abs error: 1.7547951609752197
Thresh=0.0016366612, n=91, explained variance: 0.5419937771451893, mean abs error: 1.7459400263836269
Thresh=0.0016366612, n=91, explained variance: 0.5419937771451893, mean abs error: 1.7459400263836269
Thresh=0.0016366612, n=91, explained variance: 0.5419937771451893, mean abs error: 1.7459400263836269
Thresh=0.001472995, n=93, explained variance: 0.579442669970898, mean abs error: 1

In [ ]:
nfeature_gridsearch_df.to_csv('/Users/Rohil/Documents/iGEM/yemen/feature_engineering/nfeatures_gridsearch_df_broad.csv', index = False)

In [241]:
five_thresh = thresholds[0::5]

In [242]:
nfeature_gridsearch_df = pd.DataFrame(columns=['threshold', 'n_features', 'best_params', 'explained_variance', 'mae'])

param_grid = {'nthread':[4], #when use hyperthread, xgboost may become slower
              'objective':['reg:linear'],
              'learning_rate': [0.005, 0.01, 0.05], #so called `eta` value
              'max_depth': [13],
              'min_child_weight': [30, 45],
              'silent': [1],
              'subsample': [0.4, 0.5],
              'colsample_bytree': [0.4, 0.5],
              'n_estimators': [100, 200, 300],
}

for thresh in five_thresh:
    # select features using threshold
    selection = SelectFromModel(xgb, threshold=thresh, prefit=True)
    
    selection_model = xgboost.XGBRegressor()
    
    grid_search = GridSearchCV(selection_model,
                        param_grid = param_grid,
                        cv = TimeSeriesSplit(n_splits=2).split(X_train),
                        n_jobs = -1,
                        scoring = 'explained_variance',
                        verbose=True)

    select_X_train = selection.transform(X_train)
    
    grid_search.fit(select_X_train, y_train)
    
    # eval model
    select_X_test = selection.transform(X_test)
    y_pred = grid_search.predict(select_X_test)
    score = explained_variance_score(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    
    print('Thresh=%s, n=%s, explained variance: %s, mean abs error: %s' % (thresh, select_X_train.shape[1], score, mae))
    
    nfeature_gridsearch_df = nfeature_gridsearch_df.append(pd.Series({'threshold':thresh, 'n_features':select_X_train.shape[1], 'best_params': grid_search.best_params_, 'explained_variance':score, 'mae': mae}), ignore_index = True)
    

Fitting 2 folds for each of 72 candidates, totalling 144 fits


[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    6.7s
[Parallel(n_jobs=-1)]: Done 144 out of 144 | elapsed:   15.2s finished


Thresh=0.052142724, n=1, explained variance: 0.8335351288788623, mean abs error: 1.3950294505975602
Fitting 2 folds for each of 72 candidates, totalling 144 fits


[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    7.1s
[Parallel(n_jobs=-1)]: Done 144 out of 144 | elapsed:   17.5s finished


Thresh=0.024278095, n=6, explained variance: 0.8236411588958937, mean abs error: 1.4731335042049563
Fitting 2 folds for each of 72 candidates, totalling 144 fits


...

Thresh=0.0, n=183, explained variance: 0.8401241066283188, mean abs error: 1.431569199237605
Fitting 2 folds for each of 72 candidates, totalling 144 fits


[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   15.6s
[Parallel(n_jobs=-1)]: Done 144 out of 144 | elapsed:   59.5s finished


Thresh=0.0, n=183, explained variance: 0.8401241066283188, mean abs error: 1.431569199237605
Fitting 2 folds for each of 72 candidates, totalling 144 fits


[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   15.9s
[Parallel(n_jobs=-1)]: Done 144 out of 144 | elapsed:   59.4s finished


Thresh=0.0, n=183, explained variance: 0.8401241066283188, mean abs error: 1.431569199237605


In [252]:
nfeature_gridsearch_df.to_csv('/Users/Rohil/Documents/iGEM/yemen/feature_engineering/nfeatures_gridsearch_df.csv', index = False)

In [158]:
preds = pd.Series(preds)

In [159]:
preds.index = y_test.index

In [160]:
prediction_df = pd.DataFrame(preds).merge(pd.DataFrame(full_data[['gov_iso', 'date']]), left_index = True, right_index = True)
prediction_df.columns = ['pred', 'gov_iso' ,'date']

pred_crosstab = prediction_df.pivot_table(index = 'date', columns = 'gov_iso', values = 'pred')

In [161]:
pred_crosstab.head()

gov_iso,YE-AB,YE-AD,YE-AM,YE-BA,YE-DA,YE-DH,YE-HD-AL,YE-HJ,YE-HU,YE-IB,...,YE-LA,YE-MA,YE-MR,YE-MW,YE-RA,YE-SA,YE-SD,YE-SH,YE-SN,YE-TA
date,,,,,,,,,,,,,,,,,,,,,
2017-08-14,20.710947,4.897898,29.631374,10.169798,30.773361,21.468771,1.763795,17.378012,8.233446,5.356342,...,14.901932,4.875543,2.443946,27.455353,11.457116,22.700748,4.003278,1.062638,3.486799,3.953214
2017-08-15,20.583555,4.934886,30.043766,9.911448,28.758907,21.501932,1.720262,18.624380,8.206692,5.742295,...,14.471645,4.805513,2.414942,27.462757,11.740501,25.895361,3.997659,1.190431,3.157129,3.378301
2017-08-16,20.732105,5.138100,29.280275,10.165749,29.152632,21.501354,1.601228,18.985281,8.106464,5.833121,...,12.429628,4.876575,2.514567,25.924494,12.047580,26.815323,3.698807,1.177160,3.233928,4.321141
2017-08-17,21.188223,5.064285,29.029499,15.176261,27.624121,21.160444,1.606588,19.460413,8.119987,5.795109,...,13.946028,4.937258,2.428747,25.438599,12.162300,26.941118,4.514264,1.128298,3.218765,4.857874
2017-08-18,21.926449,5.023776,28.968809,15.482224,27.531481,19.783064,1.544193,20.020969,8.407865,5.705154,...,14.573035,4.978606,2.463442,25.125048,12.514949,27.085239,4.581815,1.080929,3.924248,4.285110


In [162]:
y_test_df = y.drop('date', axis=1).merge(pd.DataFrame(full_data[['gov_iso' ,'date']]), left_index = True, right_index = True)

In [163]:
y_test_crosstab = y_test_df.pivot_table(index = 'date', columns = 'gov_iso', values = 'weekly_cases')

In [164]:
y_test_crosstab.head()

gov_iso,YE-AB,YE-AD,YE-AM,YE-BA,YE-DA,YE-DH,YE-HD-AL,YE-HJ,YE-HU,YE-IB,...,YE-LA,YE-MA,YE-MR,YE-MW,YE-RA,YE-SA,YE-SD,YE-SH,YE-SN,YE-TA
date,,,,,,,,,,,,,,,,,,,,,
2017-05-31,7.853387,13.167803,32.415759,11.652744,16.109842,9.466921,0.0,18.740863,9.986982,8.009573,...,2.625525,2.448999,0.0,23.163472,11.446652,39.580615,0.168074,0.012279,11.477125,5.678411
2017-06-01,8.405887,11.983601,33.929278,12.699393,21.715429,10.586107,0.0,19.486937,11.311516,9.337257,...,3.664075,2.632608,0.0,23.933585,14.303009,42.529852,0.180898,0.039907,12.512657,6.571019
2017-06-02,8.958386,10.799400,35.442796,13.746042,27.321017,11.705293,0.0,20.233010,12.636049,10.664942,...,4.702626,2.816217,0.0,24.703698,17.159365,45.479089,0.193723,0.067536,13.548189,7.463628
2017-06-03,9.510886,9.615199,36.956315,14.792691,32.926605,12.824479,0.0,20.979084,13.960583,11.992626,...,5.741176,2.999826,0.0,25.473811,20.015721,48.428327,0.206548,0.095164,14.583722,8.356236
2017-06-04,9.494190,6.325547,35.491447,13.803081,32.798828,12.447947,0.0,19.014812,13.478453,12.228111,...,6.698759,3.090970,0.0,23.915834,20.636611,46.302799,0.192373,0.092094,13.425218,8.417056


In [165]:
def set_style(color):
    plt.style.use(['seaborn-' + color, 'seaborn-paper'])

In [166]:
def plot_pred_against_actual(pred_crosstab, test_crosstab):
    
    set_style('white')
    
    fig, ax = plt.subplots(21,1,figsize = (6,15), sharex=True)
    
    cols = test_crosstab.columns
    
    for i in range(0,21):
        
        test_crosstab[cols[i]].plot(kind='line', ax = ax[i], label = 'true_val', legend = True, color = 'red')
                            
        pred_crosstab[cols[i]].plot(kind='line', ax = ax[i], label= 'xgb', legend = True, color = 'blue')   
          
        ax[i].legend().set_visible(False)
        ax[i].set_ylabel(cols[i])
        ax[i].yaxis.set_label_position('right')
        ax[i].spines['right'].set_visible(False)
        ax[i].spines['top'].set_visible(False)
        ax[i].spines['bottom'].set_visible(True)
        
    ax[10].legend().set_visible(True)
    ax[10].legend(fontsize=10, loc='center left', bbox_to_anchor=(1.05, 0.5))
 
    fig.subplots_adjust(hspace = .2)    
        
    fig.savefig('/Users/Rohil/Documents/iGEM/yemen/plot_xgb_all_features.png', dpi = 500, bbox_inches = 'tight')
    
    plt.close('all')

In [167]:
plot_pred_against_actual(pred_crosstab, y_test_crosstab)

In [185]:
feature_importance_df = pd.Series(xgb.feature_importances_, index = X_full.drop('date', axis=1).columns).sort_values(ascending = False)

In [69]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

Using TensorFlow backend.


In [ ]:
def train_lstm():
    model = Sequential()
    model.add(LSTM(50, input_shape=(train_X.shape[1], train_X.shape[2])))
    model.add(Dense(1))
    model.compile(loss='mae', optimizer='adam')
    # fit network
    history = model.fit(train_X, train_y, epochs=50, batch_size=72, validation_data=(test_X, test_y), verbose=2, shuffle=False)

In [ ]:

# plot history
pyplot.plot(history.history['loss'], label='train')
pyplot.plot(history.history['val_loss'], label='test')
pyplot.legend()
pyplot.show()